In [2]:
%pylab inline
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
import os
import chainer
project_path = os.getcwd() + '/../../'
sys.path.append(project_path + "utils")
sys.path.append(project_path + "data")
sys.path.append(project_path + "multi_models/cnn/")

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.grid_search import GridSearchCV
from sklearn.externals import joblib
import sklearn

Populating the interactive namespace from numpy and matplotlib


In [3]:
import seaborn as sns

/Users/admin/Desktop/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [4]:
# utils
import analysis
import function
import other
import local_bayes
import visualize

In [5]:
# data
from base import BaseDataManager
import get_normal_representation 
from preprocessing import Preprocessing
from data import Data

In [35]:
data_name_ls = ['science', 'ajax', 'java', 'news', 'css', 'javascript', 'opensource', 'webdesign', 'design', 'linux', 'photography']
raw_data_dir = project_path + 'original_data/bookmark/'
save_data_dir = project_path + 'data/datasets/bookmark_new/'
train_test_ratio = 0.8
stride = 5
balance = True
output_dim_ls = [3]

base_data_manager = BaseDataManager(raw_data_dir, save_data_dir)
D = Data(raw_data_dir, save_data_dir)
P = Preprocessing(raw_data_dir, save_data_dir)

In [36]:
for output_dim in output_dim_ls:
    
    for data_name in data_name_ls:
        print data_name
        data = base_data_manager.load_data('%s.twd' % data_name)
        ts_ls, ts_sum_ls = get_normal_representation.get_ts_normal_representation(data)
        ts_dict = base_data_manager.div_train_test(ts_ls, train_test_ratio)
        started_ts_dict = base_data_manager.get_started_ts(ts_dict)
        input_ts_dict, label_dict, target_ts_dict, input_sum_dict = \
                                    base_data_manager.make_supervised_data(started_ts_dict, stride, output_dim)
#         if balance:
#             input_ts_dict, label_dict, target_ts_dict = P.balancing_sampling(input_ts_dict, label_dict, target_ts_dict, output_dim)
        data_type = 'normal'
        D.make_train_data(input_ts_dict, label_dict, target_ts_dict, output_dim, data_type, data_name)

science
ajax
java
news
css
javascript
opensource
webdesign
design
linux
photography


In [37]:
label_dict

{'test': array([[ 1.,  1.,  1.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0.],
        [ 0.,  1.,  1.],
        [ 1.,  1.,  0.]]), 'train': array([[ 0.,  0.,  0.],
        [ 0.,  1.,  1.],
        [ 0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.],
        [ 0.,  0.,  0.]])}

## data load

In [39]:
save_data_dir = '../../data/datasets/bookmark_new/normal/'
train_ajax = numpy.load(save_data_dir + 'ajax' + '/output_dim=3/train.npz')
test_ajax = numpy.load(save_data_dir + 'ajax' + '/output_dim=3/test.npz')
target_ajax = numpy.load(save_data_dir + 'ajax' + '/output_dim=3/target.npz')

In [40]:
train_ajax['y']

array([[ 1.,  1.,  0.],
       [ 0.,  1.,  1.],
       [ 0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 1.,  1.,  0.]])

In [45]:
import chainer.functions as F

In [48]:
a = F.array.split_axis.split_axis(chainer.Variable(train_ajax['y']), 3, 1)

In [50]:
a[0].data

array([[ 1.],
       [ 0.],
       [ 0.],
       ..., 
       [ 0.],
       [ 0.],
       [ 1.]])

## train

In [41]:
import cnn_multi

In [43]:
epoch_num = 300
# batch_size = 50
for output_dim in [3]:
    save_data_dir = project_path + 'data/datasets/bookmark_new/normal/'
    save_model_dir = project_path + "results/normal/epoch_num=%s_output_dim=%s/ajax_%s/" % (epoch_num, output_dim, epoch_num)
    other.mkdirs(save_model_dir)
    train = numpy.load(save_data_dir + 'ajax' + '/output_dim=%s/train.npz' % output_dim)
    test = numpy.load(save_data_dir + 'ajax' + '/output_dim=%s/test.npz' % output_dim)
    cnntrain = cnn_multi.MultiCNN(save_model_dir=save_model_dir, epoch_num=epoch_num)
    cnntrain.train(train, test, output_dim)

epoch 1
graph generated


InvalidType: 
Invalid operation is performed in: Accuracy (Forward)

Expect: in_types[1].ndim == 1
Actual: 0 != 1

## Keras train

In [ ]:
model = Sequential()
model.add(Convolution2D(nb_filter, 3, 3, input_shape=(img_channels, img_rows, img_cols)))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filter, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))